# Depths and Snow Pit Data Package Contents

In [ ]:
# import libraries / packages
import os
from pathlib import Path
import glob
import pandas as pd
import numpy as np

#plotting imports
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use(['seaborn-notebook'])


# Download snow depth data from NSIDC
From the SnowEx20 [Depth Probe Landing Page](https://nsidc.org/data/SNEX20_SD/versions/1), you can download data and access the [User's Guide](https://nsidc.org/sites/nsidc.org/files/SNEX20_SD-V001-UserGuide_1.pdf)
<img src="images/SD-landingPage.png">

The Community Snow Depth Probe data package is a single CSV with over 36,000 geolocated snow depths! Three different instrument types were used to measure depths and are recorded in the Measurement Tool column.

Instructions:
1) Click on link to landing page
1) Go to bottom of page
1) Click on download script
1) Save to your snowex21/scripts folder locally

## Programmatically download snow depth data from NSIDC

In [ ]:
# Enter the path of the python script location
%run '../scripts/nsidc-download_SNEX20_SD.001_2021-07-13.py' 
print('Grand Mesa 2020 Snow Depth data download complete') 

In [ ]:
%%bash

# do some file organizing by moving those files to a new directory
mv *.csv ../data/depths/
mv *.xml ../data/depths/

In [ ]:
# show filename in new location:
path = Path('../data/depths/')

for filename in path.glob('*.csv'):
    print(filename.name)

### Read the Depth File

In [ ]:
# open depth data file, and create pandas dataframe
df = pd.read_csv(filename, sep=',', header=0, parse_dates=[[2,3]]) #parse the date[2] and time[3] columns such that they are read in as datetime dtypes
    
print('file has been read, and is ready to use.')

In [ ]:
# check data types for each column
df.dtypes

### Prep for Data Analysis

In [ ]:
# rename some columns for ease further down
df.rename(columns = {
    'Measurement Tool (MP = Magnaprobe; M2 = Mesa 2; PR = Pit Ruler)':'Measurement Tool', 
    'Date (yyyymmdd)_Time (hh:mm, local, MST)': "Datetime"},
          inplace = True)

# set up filter for IOP date range
start = pd.to_datetime('1/28/2020') #first day of GM IOP campaign
end   = pd.to_datetime('2/12/2020') #last day of GM IOP campaign

# filter the IOP date range
df = df[(df['Datetime'] >= start) & (df['Datetime'] <= end)]

print('DataFrame shape is: ', df.shape)
df.head()

#### Use .groupby() to sort the data set

In [ ]:
# group data by the measurement tool 
gb = df.groupby('Measurement Tool', as_index=False).mean().round(1)

# show mean snow depth from each tool
gb[['Measurement Tool', 'Depth (cm)']]

#### Find depths associated with a certain measurement tool

In [ ]:
print('List of Measurement Tools: ', df['Measurement Tool'].unique())

In [ ]:
r = df.loc[df['Measurement Tool'] == 'PR']
print('DataFrame shape is: ', r.shape)
r.head()

Let's make sure we all have the same pd.DataFrame() again

In [ ]:
# pit ruler snow depths from Grand Mesa IOP
r = df.loc[df['Measurement Tool'] == 'PR'] 
print( 'DataFrame is back to only pit ruler depths')

### Plotting

In [ ]:
# plot pit ruler depths 
ax = r.plot(x='Easting', y='Northing', c='Depth (cm)', kind='scatter', alpha=0.7, colorbar=True, colormap='PuBu', legend=True)
ax.set_title('Grand Mesa Pit Ruler Depths')
ax.set_xlabel('Easting [m]')
ax.set_ylabel('Northing [m]')
plt.show()

print('Notice the point on the far right - that is the "GML" or Grand Mesa Lodge pit where all instruments were deployed for a comparison study. pitID=GML')

In [ ]:
# plot histogram of pit ruler depths
ax = r['Depth (cm)'].plot.hist(bins=25)
ax.grid()
ax.set_title('Grand Mesa Pit Ruler Depths')
ax.set_xlabel('Snow Depth (cm)')
ax.set_ylabel('Frequency')